### **STEP ONE** OF DATA PROCESSING PIPELINE

##### This script imports data in which a Reliable Component Analysis was performed. Visual Stimulus was dual frequency tagged: 2F1 = 6 Hz, 2F2 = 7.5 Hz (inverting stim). Each participant completed 2 sessions. For each session, a xF1 and xF2 bandpass filter was performed. So 1 single participant has 4 data files. This is for a frequency-based analysis of the LTP paradigm

In [1]:
# load packages
import numpy as np 
import scipy.io
from scipy.io   import  loadmat
import pandas as pd
import os
import matplotlib.pyplot as plt #import matplotlib as plt
from scipy.optimize import curve_fit 
import seaborn as sns #import mat73
import pickle as pkl
from datetime import datetime
from itertools import chain

### Set Dir Path(s): MainDir, SaveDataDir

In [2]:
# Main Directory of processed file from MatLab
MainDir = 'C:\\plimon\\LTP_analysis\\RCA_F1\\RCA\\' # set dir - on my computer
os.chdir(MainDir) # change old dir, to this dir
d = os.listdir(MainDir) # list files in dir
print(f'Aviable Files to choose from: {len(d)}')
print(f'Files on hand: {d}')
##############################################
FileN_f1 = d[2] # choose one
FileN_f2  = d[3]                         
file_path1 = os.path.join(MainDir, FileN_f1) # join paths and prep 2 load
print('Current WD:',file_path1) # does path exist ... ?
print('Does File #1 Exist?',os.path.exists(file_path1)) # yes or no

file_path2 = os.path.join(MainDir, FileN_f2) # join paths and prep 2 load
print('Current WD:',file_path2) # does path exist ... ?
print('Does File #2 Exist?',os.path.exists(file_path1)) # yes or no

Aviable Files to choose from: 4
Files on hand: ['OfficialControlsF1_rcaResults_Sweep_contrast sweeps.mat', 'OfficialControls_F2_rcaResults_Sweep_contrast sweeps.mat', 'OfficialF1_rcaResults_Sweep_F2F4F6F8.mat', 'OfficialF2_rcaResults_Sweep_F2F4F6F8.mat']
Current WD: C:\plimon\LTP_analysis\RCA_F1\RCA\OfficialF1_rcaResults_Sweep_F2F4F6F8.mat
Does File #1 Exist? True
Current WD: C:\plimon\LTP_analysis\RCA_F1\RCA\OfficialF2_rcaResults_Sweep_F2F4F6F8.mat
Does File #2 Exist? True


In [ ]:
# # Save Data Dir ...
# #SaveDataDir = 'D:\\AttnXV3_analysis\\RCA_F1\\AllSubjSweepRCA\\' # set dir where files (.pkl, .csv) will be saved
# SaveDataDir = 'C:\\plimon\\LTP_analysis\\RCA_F1\\' # set dir where files (.pkl, .csv) will be saved
# FileOutName = 'AllSessionDataCombin_allHarms' 
# newPath = os.path.join(SaveDataDir, FileOutName)
# if not os.path.exists(SaveDataDir):
#     os.makedirs(SaveDataDir)
# print(f'Path to save data:{SaveDataDir}')
# ######################################################
# dnt = datetime.now() # add date and time bc im wreckless when saving ..
# fdnt = dnt.strftime("%Y%m%d_%H%M%S") # set the above as a string ...
# FileN = f'{FileOutName}_{fdnt}.pkl' 
# #FileNToMatlab = f'{FileOutName}_{fdnt}.h5' 
# #FileN = f'{FileOutName}_{fdnt}.csv' 
# NewFileNPath = os.path.join(SaveDataDir,FileN)
# print('Full New File Dir: ', NewFileNPath)

In [3]:
df_f1 = scipy.io.loadmat(file_path1)
df_f2 = scipy.io.loadmat(file_path2)

#### Load F1 and F2 RCA Data

In [4]:
rca_f1 = df_f1['rcaResult']['projectedData'][0,0]
f1 = [rca_f1[x,0] for x in range(rca_f1.shape[0])] # entry per subject

In [5]:
rca_f2 = df_f2['rcaResult']['projectedData'][0,0]
f2 = [rca_f2[x,0] for x in range(rca_f2.shape[0])]

In [10]:
# load subject names ...
SubNames = df_f1['rcaResult'][0,0][5]
FileName = [x[0][3:] for subjlist in SubNames for x in subjlist[0][2][0]]
#FileName = np.sort(FileName)
print(f'Total Data Files: {len(FileName)}')

Total Data Files: 66


#### Make Subj Session Name Format Uniform and extract sub ID 

In [12]:
# set all file names to the same hyphen format
FileName = [FileName.replace('_','-') for FileName in FileName]
#print(len(FileName))
#### Set some params we'll need
#[NumCols, NumComps, b] = np.shape(f1[0]) # 24 x 4 x n-Trials
NumFiles = int(len(FileName))

#print(NumCols,NumCmps, b) # new data dims 
# Find How Many Subject Names There are ...
string_ind = '-'
uniqueSubs = []
SessFileType = []
'for each file name, get sub ID and also session record'
for n in range(NumFiles):
    x = FileName[n] # single file name string ie: 'nl-xxxx_attnX'
    if string_ind in x:
        y = x.split(string_ind)[0] # get subj number
        #print(y)
        z = x.split(string_ind)[1] # get session condition name 
        #print(z)
        # from each session name extract import info that will help us index and exclude later on ..
        uniqueSubs.append(y) # store all participant numbers (repeating - will be sorted later)
        SessFileType.append(z) # store the condtion label

In [16]:
print(f'Subject IDs: {uniqueSubs}')
print(f' Conditions: {SessFileType[0:2]}')

Subject IDs: ['2651', '2651', '2652', '2652', '2653', '2653', '2654', '2654', '2655', '2655', '2657', '2657', '2658', '2658', '2659', '2659', '2660', '2661', '2661', '2663', '2663', '2664', '2664', '2665', '2665', '2666', '2666', '2667', '2667', '2668', '2668', '2669', '2669', '2670', '2670', '2671', '2671', '2672', '2672', '2674', '2674', '2676', '2677', '2677', '2678', '2695', '2695', '2696', '2696', '2697', '2697', '2708', '2715', '2716', '2726', '2727', '2728', '2728', '2733', '2734', '345202', '345202', '345215', '345215', '345216', '345216']
 Conditions: ['attnL', 'attnR']


In [25]:
# given double sessions.., how many unique participants do we have?
TotalFiles = np.array(uniqueSubs)
[NumSubs, SessCounts] = np.unique(TotalFiles, return_counts = True) # returns unique subject and how many sessions they did (should be 2)
print(f'Total Participants: {len(NumSubs)}')
[FileQuants, TotSess] = np.unique(SessCounts, return_counts = True) # returns counts of how many subs did 1 session and 2 sessions 
# print(f'{TotSess[1]} Participants completed the study')
# print(f'{TotSess[0]} Participants did not complete the study')

Total Participants: 38


In [27]:
TwoSess = SessCounts == 2 # index of who completed 2 sessions ..
GoodSubjs = NumSubs[TwoSess] # subs who completed 2 sessions 
SingleSessSubjs = NumSubs[~TwoSess] # subs who did not complete 2 sessions 
#print(f'Subjects with 2 files: {len(GoodSubjs)}')
#print(GoodSubjs)
#print()
#print(f'Single Subjs: {SingleSessSubjs}')


### If there is a specific group of subjects to look at, here is where subj ID's can be grouped

In [22]:
dataSet_Inds = ['2651','2658','2659','2661','2664','2666','2668','2672','2674','2695','345202','345215','2627','2715','2726']

#### Have to add this step because I ran someone for attnR **twice** ... yikes ...

In [28]:
# makes sure paticipants did complete attnL and attnR
FNameCrit = ['attnL', 'attnR'] # session names 
CleanSubjs = []
 
for i in range(len(GoodSubjs)):
    subj = GoodSubjs[i]
    f_list = [x for x in FileName if subj in x] # import all strongs were sub number is 
    list_check  = np.sort(f_list) # abc order strings -  attnL and THEN attnR
    counter = 0
    for n in range(len(FNameCrit)):
        if FNameCrit[n] in list_check[n]: # expt label should match file name in same position
            counter = counter + 1 # if so add 1 
            if (n == 1) and (counter == 2): # if both files strings are different, append
                CleanSubjs.append(GoodSubjs[i])
        else: # if not, add em to the singletons ... 
            print(f'{GoodSubjs[i]} did not match file name for {FNameCrit[n]}, moving subj to proper file ind array')
            str_nm = (np.array([GoodSubjs[i]], dtype=object))
            SingleSubs = np.concatenate((str_nm, SingleSessSubjs))

2663 did not match file name for attnL, moving subj to proper file ind array


#### How many participants **actually** completed both sessions:

In [29]:
print(f'{len(CleanSubjs)} Participants Completed AttnL and AttnR')
print(f'{len(SingleSubs)} Participants did not complete both sessions')
print(CleanSubjs)

27 Participants Completed AttnL and AttnR
11 Participants did not complete both sessions
['2651', '2652', '2653', '2654', '2655', '2657', '2658', '2659', '2661', '2664', '2665', '2666', '2667', '2668', '2669', '2670', '2671', '2672', '2674', '2677', '2695', '2696', '2697', '2728', '345202', '345215', '345216']


### start workbench 

In [ ]:
All_Data_sIn = CleanSubjs[26]
print(f'importing subject {All_Data_sIn}s data')
# find position of both files located
SubjFile_Pos = [(i, x) for i, x in enumerate(FileName) if All_Data_sIn in x]
# only get the number position
fx_fileInds = np.array([SubjFile_Pos[0][0],SubjFile_Pos[1][0]])
print(f'File Pos: {fx_fileInds}')
# double check!!
'This dict contains Attention-Induction Congruent F1 and F2 data and Atention-Induction Incongruent F1,F2 data Unprocessed',
DuoSessionData = [f1[fx_fileInds[0]], f2[fx_fileInds[0]],f1[fx_fileInds[1]], f2[fx_fileInds[1]]]
print(f'{FileName[fx_fileInds[0]]},{FileName[fx_fileInds[1]]}')

In [ ]:
All_Data_sIn = CleanSubjs[26]
print(f'importing subject {All_Data_sIn}s data')
# find position of both files located
SubjFile_Pos = [(i, x) for i, x in enumerate(FileName) if All_Data_sIn in x]
# only get the number position
fx_fileInds = np.array([SubjFile_Pos[0][0],SubjFile_Pos[1][0]])
print(f'File Pos: {fx_fileInds}')
# double check!!
'This dict contains Attention-Induction Congruent F1 and F2 data and Atention-Induction Incongruent F1,F2 data Unprocessed',
# DuoSessionData = [f1[fx_fileInds[0]], f1[fx_fileInds[1]],f2[fx_fileInds[0]], f2[fx_fileInds[1]]]
DuoSessionData = [f1[fx_fileInds[0]], f2[fx_fileInds[0]],f1[fx_fileInds[1]], f2[fx_fileInds[1]]]
# print(f'{FileName[fx_fileInds[0]]},{FileName[fx_fileInds[1]]}')
rca_comp = 0
ProcessedConditionData = {} # pcd processed condition data
diffs ={}

for datamat in range(4):

    ProcessedConditionData[datamat] = {}
    diffs[datamat] ={}
    #print(DuoSessionData[i].shape)
    x = np.array(DuoSessionData[datamat])
    # look at first RCA component
    xf = x[:,rca_comp,:] # [48 x 78] 1f1 2f1 4f1 6f1

    c,r = np.shape(xf) # get dims of data 
    ci = int(c/2) # real and imaginary values to divide
    ri = int(r/2) # pre post boundries to divide 
    ComplexData = np.zeros((2,ci,r)) #real/imag x contrast/harmonic vals x trials

    realData= xf[:ci,:] # real values for all trials 
    phaseData = xf[ci:,:] # imaginary values for all trials

    # store data in one more consise array (reshape) 
    for i in range(2):
        if i == 0:
            ComplexData[0,:,:] = realData
        else:
            ComplexData[1,:,:] = phaseData

    # get all harmonic data in a single dict
    harmonicData = {}
    # harmonic epochs
    HamronicIndicies = np.arange(0,ci,6) # index to get 

    for j in range(4):
        s = HamronicIndicies[j]
        e = s+6
        harmonicData[j] = ComplexData[:,s:e,:]  #[2 x 6 x ~80 ]
        
    
    complexValues = {} # no combination of real x imaginary
    CleanCRF = {} # contrast response functions

    for l in range(4):
        hd = harmonicData[l] # 2 x 6 x numtrials

        p1 = hd[:,:,:ri] # pre  [2x6xnumtrials]
        p2 = hd[:,:,ri:] # post 

        px_set = [p1,p2] # store pre post in temp array

        cplxValsRec = np.zeros((2,2,6)) # pre/postx real/imaginary values x per contrast sweep incriment
        crfValsRec = np.zeros((2,6))

        for pp in range(2):
            dIn = px_set[pp] # import pre or post data
            for sweep in range(6): # import data for 1 contrast only 
                avgVal = dIn[:,sweep,:] # 2 x 39 vals
                av = np.nanmean(avgVal,axis = 1)
                crfValsRec[pp,sweep] = np.hypot(av[0],av[1])

            cplxValsRec[pp,:,sweep] = av
        #crf_diff = (crfValsRec[1,:] - crfValsRec[0,:])
        pp_diff = (crfValsRec[1,:] - crfValsRec[0,:]) / (crfValsRec[0,:] + crfValsRec[1,:])


        complexValues[l] = cplxValsRec

        #CleanCRF[l] = crfValsRec

        ProcessedConditionData[datamat][l] = crfValsRec

        diffs[datamat][l] = pp_diff
    
    # for k in range(1):
    #     crf = CleanCRF[k]
    #     cdata = complexValues[k]

    
# # custom_params = {"axes.spines.right": False, "axes.spines.top": False}
# sns.set_theme(context='notebook', style='darkgrid', palette='deep', font='sans-serif', font_scale=1, color_codes=True, rc=None)
fig,pxs = plt.subplots(1,4, figsize  = (15,4), sharey = True)
plt.suptitle(f'Subject #{All_Data_sIn}')
pig,fxs  = plt.subplots(figsize = (5,5))
plt.suptitle(f'Subject #{All_Data_sIn} Post Induction Response')

labels = ['A-I C','A-I Control','A-I INC','~A-I']
for p in range(4):
    crfData = ProcessedConditionData[p][0]
    d_diffs = diffs[p][0]

    pxs[p].plot(crfData[0,:], label = f'pre {labels[p]}', color = 'black')
    pxs[p].plot(crfData[1,:], label = f'post {labels[p]}', color = 'blue')
    pxs[p].hlines(0,xmin = 0, xmax = 5, color = 'black',linewidth = 4)
    pxs[p].plot(d_diffs, label = f'Net Inc.', color = 'red')
    pxs[p].legend(loc  = 'upper left')


    fxs.plot(d_diffs, label = f'{labels[p]}', linewidth = 8)
    fxs.hlines(0,xmin = 0, xmax = 5, color = 'black',linewidth = 4)
    fxs.legend()


In [ ]:
rca_comp = 0
str_split = '-'
# All_Data_sIn = SingleSessSubjs[6]
for sIn in range(len(SingleSessSubjs)):
    All_Data_sIn = SingleSessSubjs[sIn]
    #SingleSessSubjs or CleanSubjs

    print(f'importing subject {All_Data_sIn}s data')
    SubjFile_Pos = [(i, x) for i, x in enumerate(FileName) if All_Data_sIn in x]
    # only get the number position
    'this code chunk deals with incomplete data by providing and creating  matrix of nans'
    if (len(SubjFile_Pos)) == 1:
        'make emoty df for missing values to get pretty plots still '
        emptyMat_df1 = np.zeros((48,4,78))
        emptyMat_df2 = np.zeros_like((emptyMat_df1))
        emptyMat_df1[:] - np.nan
        emptyMat_df2[:] - np.nan

        print(f'only 1 file found')
        'locate file position only - contains file name too'
        fx_fileInds = np.array([SubjFile_Pos[0][0]])
        SessionCond = SubjFile_Pos[0][1]
        #print(SessionCond)
        'determine what data is missing (attnL/R) and place data in correct format'
        Expt = SessionCond.split(str_split)[1]
        if Expt == 'attnL':
            #'import first data poitions and set latter conditions as missing data'
            print(f'AttnL File Pos: {fx_fileInds}, Exp Session: {SessionCond}')

            DuoSessionData = [f1[fx_fileInds[0]], f2[fx_fileInds[0]],emptyMat_df1,emptyMat_df2]

        elif Expt == 'attnR':
            #'import first data poitions as missing data and import last data positions as missing data'
            print(f'AttnR File Pos: {fx_fileInds}, Exp Session: {SessionCond}')

            DuoSessionData = [emptyMat_df1,emptyMat_df2,f1[fx_fileInds[0]], f2[fx_fileInds[0]]]


    # 'If subject completed both data files, divvy data'
    else:
        print(f'2 files found')
        fx_fileInds = np.array([SubjFile_Pos[0][0],SubjFile_Pos[1][0]])
        print(f'File Pos: {fx_fileInds}')
        'This dict contains Attention-Induction Congruent F1 and F2 data and Atention-Induction Incongruent F1,F2 data Unprocessed'

        DuoSessionData = [f1[fx_fileInds[0]], f2[fx_fileInds[0]],f1[fx_fileInds[1]], f2[fx_fileInds[1]]]
        #print(f'{FileName[fx_fileInds[0]]},{FileName[fx_fileInds[1]]}')


    ProcessedConditionData = {} # pcd processed condition data
    diffs ={}
    for datamat in range(4):

        ProcessedConditionData[datamat] = {}
        diffs[datamat] ={}
        #print(DuoSessionData[i].shape)
        x = np.array(DuoSessionData[datamat])
        # look at first RCA component
        xf = x[:,rca_comp,:] # [48 x 78] 1f1 2f1 4f1 6f1

        c,r = np.shape(xf) # get dims of data 
        ci = int(c/2) # real and imaginary values to divide
        ri = int(r/2) # pre post boundries to divide 
        ComplexData = np.zeros((2,ci,r)) #real/imag x contrast/harmonic vals x trials

        realData= xf[:ci,:] # real values for all trials 
        phaseData = xf[ci:,:] # imaginary values for all trials

        # store data in one more consise array (reshape) 
        for i in range(2):
            if i == 0:
                ComplexData[0,:,:] = realData
            else:
                ComplexData[1,:,:] = phaseData

        # get all harmonic data in a single dict
        harmonicData = {}
        # harmonic epochs
        HamronicIndicies = np.arange(0,ci,6) # index to get 

        for j in range(4):
            s = HamronicIndicies[j]
            e = s+6
            harmonicData[j] = ComplexData[:,s:e,:]  #[2 x 6 x ~80 ]
            
        
        complexValues = {} # no combination of real x imaginary
        CleanCRF = {} # contrast response functions

        for l in range(4):
            hd = harmonicData[l] # 2 x 6 x numtrials

            p1 = hd[:,:,:ri] # pre  [2x6xnumtrials]
            p2 = hd[:,:,ri:] # post 

            px_set = [p1,p2] # store pre post in temp array

            cplxValsRec = np.zeros((2,2,6)) # pre/postx real/imaginary values x per contrast sweep incriment
            crfValsRec = np.zeros((2,6))

            for pp in range(2):
                dIn = px_set[pp] # import pre or post data
                for sweep in range(6): # import data for 1 contrast only 
                    avgVal = dIn[:,sweep,:] # 2 x 39 vals
                    av = np.nanmean(avgVal,axis = 1)
                    crfValsRec[pp,sweep] = np.hypot(av[0],av[1])

                cplxValsRec[pp,:,sweep] = av
            #crf_diff = (crfValsRec[1,:] - crfValsRec[0,:])
            pp_diff = (crfValsRec[1,:] - crfValsRec[0,:]) / (crfValsRec[0,:] + crfValsRec[1,:])


            complexValues[l] = cplxValsRec

            #CleanCRF[l] = crfValsRec

            ProcessedConditionData[datamat][l] = crfValsRec

            diffs[datamat][l] = pp_diff
        
        # for k in range(1):
    fig,pxs = plt.subplots(1,4, figsize  = (15,4), sharey = True)
    plt.suptitle(f'Subject #{All_Data_sIn}')
    pig,fxs  = plt.subplots(figsize = (5,5))
    plt.suptitle(f'Subject #{All_Data_sIn} Post Induction Response')

    labels = ['A-I C','A-I Control','A-I INC','~A-I']
    for p in range(4):
        crfData = ProcessedConditionData[p][0]
        d_diffs = diffs[p][0]

        pxs[p].plot(crfData[0,:], label = f'pre {labels[p]}', color = 'black')
        pxs[p].plot(crfData[1,:], label = f'post {labels[p]}', color = 'blue')
        pxs[p].hlines(0,xmin = 0, xmax = 5, color = 'black',linewidth = 4)
        pxs[p].plot(d_diffs, label = f'Net Inc.', color = 'red')
        pxs[p].legend(loc  = 'upper left', fontsize = 8)

        fxs.plot(d_diffs, label = f'{labels[p]}', linewidth = 8)
        fxs.hlines(0,xmin = 0, xmax = 5, color = 'black',linewidth = 4)
        fxs.legend()

In [ ]:
# double check!!
'This dict contains Attention-Induction Congruent F1 and F2 data and Atention-Induction Incongruent F1,F2 data Unprocessed',
DuoSessionData = [f1[fx_fileInds[0]], f1[fx_fileInds[1]],f2[fx_fileInds[0]], f2[fx_fileInds[1]]]
print(f'{FileName[fx_fileInds[0]]},{FileName[fx_fileInds[1]]}')
# rca_comp = 0
ProcessedConditionData = {} # pcd processed condition data
diffs ={}

for datamat in range(4):

    ProcessedConditionData[datamat] = {}
    diffs[datamat] ={}
    #print(DuoSessionData[i].shape)
    x = np.array(DuoSessionData[datamat])
    # look at first RCA component
    xf = x[:,rca_comp,:] # [48 x 78] 1f1 2f1 4f1 6f1

    c,r = np.shape(xf) # get dims of data 
    ci = int(c/2) # real and imaginary values to divide
    ri = int(r/2) # pre post boundries to divide 
    ComplexData = np.zeros((2,ci,r)) #real/imag x contrast/harmonic vals x trials

    realData= xf[:ci,:] # real values for all trials 
    phaseData = xf[ci:,:] # imaginary values for all trials

    # store data in one more consise array (reshape) 
    for i in range(2):
        if i == 0:
            ComplexData[0,:,:] = realData
        else:
            ComplexData[1,:,:] = phaseData

    # get all harmonic data in a single dict
    harmonicData = {}
    # harmonic epochs
    HamronicIndicies = np.arange(0,ci,6) # index to get 

    for j in range(4):
        s = HamronicIndicies[j]
        e = s+6
        harmonicData[j] = ComplexData[:,s:e,:]  #[2 x 6 x ~80 ]
        
    
    complexValues = {} # no combination of real x imaginary
    CleanCRF = {} # contrast response functions

    for l in range(4):
        hd = harmonicData[l] # 2 x 6 x numtrials

        p1 = hd[:,:,:ri] # pre  [2x6xnumtrials]
        p2 = hd[:,:,ri:] # post 

        px_set = [p1,p2] # store pre post in temp array

        cplxValsRec = np.zeros((2,2,6)) # pre/postx real/imaginary values x per contrast sweep incriment
        crfValsRec = np.zeros((2,6))

        for pp in range(2):
            dIn = px_set[pp] # import pre or post data
            for sweep in range(6): # import data for 1 contrast only 
                avgVal = dIn[:,sweep,:] # 2 x 39 vals
                av = np.nanmean(avgVal,axis = 1)
                crfValsRec[pp,sweep] = np.hypot(av[0],av[1])

            cplxValsRec[pp,:,sweep] = av
        #crf_diff = (crfValsRec[1,:] - crfValsRec[0,:])
        pp_diff = (crfValsRec[1,:] - crfValsRec[0,:]) / (crfValsRec[0,:] + crfValsRec[1,:])


        complexValues[l] = cplxValsRec

        #CleanCRF[l] = crfValsRec

        ProcessedConditionData[datamat][l] = crfValsRec

        diffs[datamat][l] = pp_diff
    
    # for k in range(1):
    #     crf = CleanCRF[k]
    #     cdata = complexValues[k]

    
# # custom_params = {"axes.spines.right": False, "axes.spines.top": False}
# sns.set_theme(context='notebook', style='darkgrid', palette='deep', font='sans-serif', font_scale=1, color_codes=True, rc=None)
fig,pxs = plt.subplots(1,4, figsize  = (15,4), sharey = True)
plt.suptitle(f'Subject #{All_Data_sIn}')
pig,fxs  = plt.subplots(figsize = (5,5))
plt.suptitle(f'Subject #{All_Data_sIn} Post Induction Response')

labels = ['A-I C','A-I Control','A-I INC','~A-I']
for p in range(4):
    crfData = ProcessedConditionData[p][0]
    d_diffs = diffs[p][0]

    pxs[p].plot(crfData[0,:], label = f'pre {labels[p]}', color = 'black')
    pxs[p].plot(crfData[1,:], label = f'post {labels[p]}', color = 'blue')
    pxs[p].hlines(0,xmin = 0, xmax = 5, color = 'black',linewidth = 4)
    pxs[p].plot(d_diffs, label = f'Net Inc.', color = 'red')
    pxs[p].legend(loc  = 'upper left')


    fxs.plot(d_diffs, label = f'{labels[p]}', linewidth = 8)
    fxs.hlines(0,xmin = 0, xmax = 5, color = 'black',linewidth = 4)
    fxs.legend()

### end workbench

#### Store Data Files as AttnL and AttnR in order

In [ ]:
FNameCrit = ['attnL', 'attnR']
FilePos = np.zeros((len(CleanSubjs), 2))

for name in range(len(CleanSubjs)):
    yIn = CleanSubjs[name] # import single subject who completed 2 sessions 
    all_files_avil = [x for x in FileName if yIn in x] # list
    all_files_avil = np.sort(all_files_avil) # might not be necessary but jic ...
    pos = [loc for loc, file in enumerate(FileName) if file in all_files_avil] # index position of files if files match attnL and attnR
    FilePos[name,:] = pos

In [ ]:
# SessNumIn = 8 # enter subj number
NumHarms = 4 # 2f2 4f1 6f2 8f1
rca_comp = 0 # enter rca component 
# set string to array \
for SessNumIn in range(len(f1)):
    x = np.array(f2[SessNumIn])
    # look at first RCA component
    xf = x[:,rca_comp,:] # [48 x 78] 1f1 2f1 4f1 6f1
    c,r = np.shape(xf) # get dims of data 
    ci = int(c/2) # real and imaginary values to divide
    ri = int(r/2) # pre post boundries to divide 
    ComplexData = np.zeros((2,ci,r)) #pre post x  real/imag x trials

    realData= xf[:ci,:] # real values for all trials 
    phaseData = xf[ci:,:] # imaginary values for all trials

    # store data in one more consise array (reshape)
    for i in range(2):
        if i == 0:
            ComplexData[i,:,:] = realData
        else:
            ComplexData[i,:,:] = phaseData

    # get all harmonic data in a single dict
    harmonicData = {}
    HamronicIndicies = np.arange(0,ci,6) # index to get 
    for j in range(4):
        s = HamronicIndicies[j]
        e = s+6
        harmonicData[j] = ComplexData[:,s:e,:] 
    complexValues = {}
    CleanCRF = {}
    for l in range(4):
        hd = harmonicData[l] # 2 x 6 x numtrials

        p1 = hd[:,:,:ri] # pre 
        p2 = hd[:,:,ri:] # post 

        px_set = [p1,p2]

        cplxValsRec = np.zeros((2,2,6)) # pre/postx real/imaginary values x per contrast sweep incriment
        crfValsRec = np.zeros((2,6))

        for pp in range(2):
            dIn = px_set[pp] # import pre or post data
            for sweep in range(6): # import data for 1 contrast only 
                avgVal = dIn[:,sweep,:] # 2 x 39 vals
                av = np.nanmean(avgVal,axis = 1)
                crfValsRec[pp,sweep] = np.hypot(av[0],av[1])
            cplxValsRec[pp,:,sweep] = av
        crf_diff = (crfValsRec[1,:] - crfValsRec[0,:])

        complexValues[l] = cplxValsRec
        CleanCRF[l] = crfValsRec
    for k in range(1):
        crf = CleanCRF[k]
        cdata = complexValues[k]

        fig,axs = plt.subplots(1,2,figsize =(6,3),sharey = False)
        axs[0].plot(crf[0,:], label = 'pre', color = 'black')
        axs[0].plot(crf[1,:], label = 'post', color = 'blue')
        axs[0].plot(crf[1,:] - crf[0,:], label = 'post increase  ', color = 'red')
        axs[0].hlines(0, xmin = 0, xmax = 5, color = 'black', linestyles = 'dashed')
        axs[0].legend(loc = 'upper left', fontsize = 6)

        axs[1].scatter(cdata[0,0,:],cplxValsRec[0,1,:], label = 'pre complexs', linewidth  = 6, color = 'black')
        axs[1].scatter(cdata[1,0,:],cplxValsRec[1,1,:], label = 'post complexs',linewidth  = 2 )
        axs[1].hlines(0, xmin = -2, xmax = 2, color = 'black', linestyles = 'dashed')
        axs[1].vlines(0, ymin = -1, ymax = 1, color = 'black', linestyles = 'dashed')
        axs[1].legend(fontsize= 5)

#### Create a big for loop to save all this data

In [ ]:
AttnL =  {'F1': [], 'F2': []} 
AttnR =  {'F1': [], 'F2': []} 
#AttnX =  {'F1': [], 'F2': []} 

# run a different file we imported (2 F1 and F2 filtered data) to save all in the same file
for iter in range(NumFiles):
    for ind in range(len(CleanSubjs)):
        attnL_FilePos = int(FilePos[ind,0])
        attnR_FilePos = int(FilePos[ind,1])
        if iter == 0:
            data = f1
            AttnL['F1'].append(data[attnL_FilePos])  # Append value to list in 'F1' key
            AttnR['F1'].append(data[attnR_FilePos])  # Append value to list in 'F1' key
        elif iter == 1:
            data = f2
            AttnL['F2'].append(data[attnL_FilePos])  # Append value to list in 'F2' key
            AttnR['F2'].append(data[attnR_FilePos])  # Append value to list in 'F2' key

### Now Saving Subjects data who only completed 1 session 

In [ ]:
SingleFiles_arr = []
for file in range(len(SingleSubs)):
    subIn = SingleSubs[file] # import one file at a time
    files_avil = [x for x in FileName if subIn in x]
    SingleFiles_arr.append(files_avil)

SinglefileNames = np.array(list(chain(*SingleFiles_arr))) #all single session names flattened
print(SinglefileNames)

In [ ]:
string_catch_L = 'attnL'
string_catch_R = 'attnR'
single_sess_ind = np.zeros(len(SinglefileNames)) # size of single sessions available
SingleSessSubName = [] # save subject name 
single_sess_pos = [] # find and store the data index 

for sInFName in range(len(SinglefileNames)):
    # determine whether its attnr or attnL
    fIn = SinglefileNames[sInFName]
    #find the postion of file in the data to organize later
    pos = [posi for posi, file in enumerate(FileName) if file in fIn]
    single_sess_pos.append(pos)

    x = fIn.split(string_ind)[1]
    y = fIn.split(string_ind)[0]
    SingleSessSubName.append(y)
    # make array to findex what files are attnL and attnR
    if string_catch_L in x:
        single_sess_ind[sInFName] = 1 # attnL ind == 1
    elif string_catch_R in x:
        single_sess_ind[sInFName] = 0 # attnL ind == 0

single_sess_pos = np.array(single_sess_pos)
# print(single_sess_pos)
# print(single_sess_ind)
# print(SingleSessSubName)

#### Index and save singles sessions as a seperate dict to export in pkl file

In [ ]:
single_sess_AttnL =  {'F1': [], 'F2': []} 
single_sess_AttnR =  {'F1': [], 'F2': []} 
l_subs = []
r_subs = []

for file_op in range(NumFiles):
    for oneSess in range(len(SinglefileNames)):
        sIn = SingleSessSubName[oneSess] # single sub names
        AttnXCond = single_sess_ind[oneSess] # condtion they did
        DataPos = single_sess_pos[oneSess] # position of data file is 
        DataPos = int(DataPos[0]) 

        if file_op == 0:
            dataIn = f1 # switch files 2 combine them
            if AttnXCond == 1:
                single_sess_AttnL['F1'].append(dataIn[DataPos]) # save data in this dict
                l_subs.append(sIn) # save subject name in this dict
            else:
                single_sess_AttnR['F1'].append(dataIn[DataPos])
                r_subs.append(sIn)

        elif file_op == 1:
            dataIn = f2 # switch files 2 combine them
            if AttnXCond == 1:
                single_sess_AttnL['F2'].append(dataIn[DataPos])
            else:
                single_sess_AttnR['F2'].append(dataIn[DataPos])


In [ ]:
SingleSessDataOut = {}

SingleSessDataOut[0] = single_sess_AttnL['F1']
SingleSessDataOut[1] = single_sess_AttnL['F2']
SingleSessDataOut[2] = single_sess_AttnR['F1']
SingleSessDataOut[3] = single_sess_AttnR['F2']

SingleSessDataOut['AttnLSubNames'] = np.array(l_subs)
SingleSessDataOut['AttnRSubNames'] = np.array(r_subs)
#SingleSessDataOut['DataNotes'] = ['keys: 0&1 attnL[f1/f2] and 2&3 attR[f1/f2], single session data']
print(SingleSessDataOut.keys())

In [ ]:
SessDataOut = {}

SessDataOut[0] = AttnL['F1']
SessDataOut[1] = AttnL['F2']
SessDataOut[2] = AttnR['F1']
SessDataOut[3] = AttnR['F2']

SessDataOut['FullSessSubjNames'] = CleanSubjs
SessDataOut['DataNotes'] = ['keys: 0&1 attnL[f1/f2] and 2&3 attR[f1/f2]']
print(SessDataOut.keys())

#### Save Data into .pkl file

In [ ]:
dataOut = {}

dataOut[0] = SessDataOut
dataOut[1] = SingleSessDataOut

In [ ]:
saveFile = 'y'

if saveFile == 'y':
 with open(NewFileNPath, 'wb') as file:
    pkl.dump(dataOut, file, protocol=pkl.HIGHEST_PROTOCOL)
    print('Sorted Data Saved! :))')
else:
    print('Did Not Save File! Change file name before switching to y!')